In [1]:
print('checking your environment works fine...')


checking your environment works fine...


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import re
import pandas as pd

In [82]:
def clean_markdown_content(content):
    # Remove links
    link_pattern = r'\[([^\[]+)\]\(([^\)]+)\)'
    content = re.sub(link_pattern, r'\1', content)

    # Remove images
    image_pattern = r'\!\[([^\[]*)\]\(([^\)]+)\)'
    content = re.sub(image_pattern, '', content)

    return content

# Usage
# cleaned_content = clean_markdown_content(markdown_content)

In [83]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from dotenv import dotenv_values

# Load environment variables from .env file
if os.path.exists(".env"):
    load_dotenv(override=True)
    config = dotenv_values(".env")

openai_client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
)

In [84]:
# curl https://open-ai-sweden-central.openai.azure.com/openai/deployments/deployment-text-embedding-3-large/embeddings?api-version=2024-02-01 \
#     -H 'Content-Type: application/json' \
#     -H 'api-key: 9bb7099f35d2452e99e970c551200935' \
#     -d '{"input": "Sample Document goes here"}'

def get_embeddings_from_azure_text_embedding(text):

    response = openai_client.embeddings.create(
        input=text,
        model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
    )

    embedding = response.data[0].embedding
    print(embedding)

    return embedding

In [86]:
import uuid
import re

def create_chunks(input_directory, output_directory):
    i=0
    # Loop through each file in the directory
    for filename in os.listdir(input_directory):
        # Check if the file is a markdown file
        if filename.endswith('.md'):
            # Open the file
            with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
                print(filename)
                # Read the file content
                content = file.read()
                tokens = num_tokens_from_string(content, "cl100k_base")
                print(tokens)

                # Extract the title, description, and date
                page_title = re.search(r'title: (.*)', content).group(1).replace('"', '')
                page_description = re.search(r'description: (.*)', content).group(1)
                page_date = re.search(r'ms.date: (.*)', content).group(1)
                
                # Split the content into chunks based on '##'
                chunks = content.split('\n## ')[1:]  # Skip the first chunk as it contains the title, description, and date
                
                # Add the chunks to the list along with the title, description, and date
                for chunk in chunks:
                    i=i+1
                    chunk_content = clean_markdown_content(chunk.strip())
                    vector = get_embeddings_from_azure_text_embedding(chunk_content)
                    
                    chunk = {
                        "id": str(uuid.uuid4()),
                        'page_title': page_title,
                        'page_description': page_description,
                        'page_date': page_date,
                        'chunk_title': chunk.split('\n')[0],  # The first line after '##' is the title of the chunk
                        'chunk_content': chunk_content,  # Remove leading and trailing whitespaces
                        'vector': vector
                    }
                    # write chunk into JSON file into output directory
                    with open(f'{output_directory}/chunk_{i}_{page_title}.json', 'w') as f:
                        json.dump(chunk, f)

In [30]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [71]:
num_tokens_from_string("tiktoken is great!", "cl100k_base")

6

In [87]:
# Define the directory where the markdown files are located
input_directory = './data/'
output_directory = './data/chunks/'
# directory = 'Z:\projects\docs-md-all'

create_chunks(input_directory, output_directory)

access-private-cluster.md
1657
[-0.012864679098129272, -0.03675263375043869, 0.00032110634492710233, -0.006448050960898399, -0.0006689224392175674, -0.04205688089132309, -0.006429197266697884, 0.06269567459821701, 0.023743407800793648, -0.007258770987391472, 0.01190941222012043, -0.0225996021181345, -0.02056337520480156, -0.018062084913253784, 0.0441182442009449, 0.008346015587449074, -0.03627500310540199, 0.010049155913293362, -0.03318295255303383, -0.033685725182294846, 0.00983547791838646, 0.02883397601544857, -0.007912375032901764, 0.020689068362116814, 0.01484434399753809, 0.000538123189471662, -0.000756514782551676, 0.025352278724312782, -0.02092788554728031, -0.02732565999031067, 0.04024690017104149, 0.015435101464390755, 0.004534375388175249, -0.013285750523209572, 0.0576176755130291, -0.010086863301694393, -0.00023017531202640384, -0.009885754436254501, -0.03680291026830673, -0.034238774329423904, -0.041880909353494644, -0.032479073852300644, 0.028004402294754982, -0.008063206

In [14]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [49]:
for i, chunk in enumerate(chunks):
    num_tokens = num_tokens_from_string(chunk['chunk_content'], "cl100k_base")
    num_tokens_json = num_tokens_from_string(json.dumps(chunk, ensure_ascii=False), "cl100k_base")
    
    print(f'chunk {i} : {num_tokens}')
    print(f'chunk {i} : {num_tokens_json}')

chunk 0 : 214
chunk 0 : 31541
chunk 1 : 914
chunk 1 : 32299
chunk 2 : 44
chunk 2 : 31352
chunk 3 : 100
chunk 3 : 31454
chunk 4 : 240
chunk 4 : 31569
chunk 5 : 79
chunk 5 : 31391
chunk 6 : 310
chunk 6 : 31724
chunk 7 : 458
chunk 7 : 31795
chunk 8 : 42
chunk 8 : 31372
chunk 9 : 44
chunk 9 : 31373
chunk 10 : 77
chunk 10 : 31440
chunk 11 : 65
chunk 11 : 31384
chunk 12 : 368
chunk 12 : 31674
chunk 13 : 458
chunk 13 : 31821
chunk 14 : 42
chunk 14 : 31373
chunk 15 : 44
chunk 15 : 31381
chunk 16 : 89
chunk 16 : 31395
chunk 17 : 151
chunk 17 : 31506
chunk 18 : 28
chunk 18 : 31355
chunk 19 : 44
chunk 19 : 31375
chunk 20 : 314
chunk 20 : 31648
chunk 21 : 98
chunk 21 : 31413
chunk 22 : 402
chunk 22 : 31794
chunk 23 : 95
chunk 23 : 31467
chunk 24 : 46
chunk 24 : 31401
chunk 25 : 751
chunk 25 : 32116
chunk 26 : 626
chunk 26 : 32039
chunk 27 : 1706
chunk 27 : 33199
chunk 28 : 49
chunk 28 : 31384
chunk 29 : 100
chunk 29 : 31445
chunk 30 : 315
chunk 30 : 31657
chunk 31 : 98
chunk 31 : 31413
chunk 32 : 

In [20]:
pip install python-dotenv azure-search-documents azure-identity

Note: you may need to restart the kernel to use updated packages.


By default, the length of the embedding vector will be `1536` for `text-embedding-3-small` or `3072` for `text-embedding-3-large`. You can reduce the dimensions of the embedding by passing in the dimensions parameter without the embedding losing its concept-representing properties.

In [88]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    SearchField,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticSearch,
    SemanticField
)

# Set your Azure Cognitive Search service endpoint and API key
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_service_api_key = os.getenv("AZURE_SEARCH_SERVICE_ADMIN_KEY")

# Create a SearchIndexClient instance
credential = AzureKeyCredential(search_service_api_key)

index_name = "index-chunks"

search_index_client = SearchIndexClient(
    endpoint=search_service_endpoint, index_name=index_name, credential=credential
)


def create_or_update_index(name: str):
    fields = [
        SimpleField(
            name="id",
            type=SearchFieldDataType.String,
            key=True,
            sortable=True,
            filterable=True,
            facetable=True,
        ),
        SearchableField(name="page_title", type=SearchFieldDataType.String),
        SearchableField(name="page_description", type=SearchFieldDataType.String),
        SearchableField(name="page_date", type=SearchFieldDataType.String),
        SearchableField(name="chunk_title", type=SearchFieldDataType.String),
        SearchableField(name="chunk_content", type=SearchFieldDataType.String),
        SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=3072, #1536,
            vector_search_profile_name="myHnswProfile",
        ),
    ]

    # Configure the vector search configuration  
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw"
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
            )
        ]
    )

    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="page_title"),
            # keywords_fields=[SemanticField(field_name="category")],
            content_fields=[SemanticField(field_name="chunk_content")]
        )
    )

    # Create the semantic settings with the configuration
    semantic_search = SemanticSearch(configurations=[semantic_config])

    # Create the search index with the semantic settings
    search_index = SearchIndex(name=name, fields=fields,
                        vector_search=vector_search, semantic_search=semantic_search)
    result = search_index_client.create_or_update_index(search_index)
    print(f' {result.name} created')

    # cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
    # scoring_profile = ScoringProfile(name="MyProfile")
    # scoring_profiles = []
    # scoring_profiles.append(scoring_profile)
    
    # index = SearchIndex(
    #     name=name,
    #     fields=fields,
    #     scoring_profiles=scoring_profiles,
    #     cors_options=cors_options,
    # )

    # result = index_client.create_or_update_index(index=index)

In [80]:
create_or_update_index(index_name)

 index-chunks created


In [112]:
def delete_index(name: str):
    index_client.delete_index(name)

In [132]:
delete_index(index_name)

In [21]:
import uuid
from azure.search.documents import SearchClient

azure_search_service_endpoint=os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

search_client = SearchClient(endpoint=azure_search_service_endpoint, index_name=index_name, credential=credential)

def upload_sample_document():
    document = {
        "id": "c130c588-c536-4569-a46b-f119285b9c40",
        "page_title": "Access a private Azure Kubernetes Service (AKS) cluster",
        "page_description": "Learn how to access a private Azure Kubernetes Service (AKS) cluster using the Azure CLI or Azure portal.",
        "page_date": "09/15/2023",
        "chunk_title": "Before you begin",
        "chunk_content": "Before you begin\n\nBefore you begin, make sure you have the following resources and permissions:\n\n* An existing private cluster. If you don't have one, see Create a private AKS cluster.\n* The Azure CLI version 2.24.0 or later. Run `az --version` to find the version. If you need to install or upgrade, see Install Azure CLI.\n* Access to the `Microsoft.ContainerService/managedClusters/runcommand/action` and `Microsoft.ContainerService/managedclusters/commandResults/read` roles on the cluster.\n\n### Limitations\n\nThe pod created by the `run` command provides `helm` and the latest compatible version of `kubectl` for your cluster with `kustomize`.\n\n`command invoke` runs the commands from your cluster, so any commands run in this manner are subject to your configured networking restrictions and any other configured restrictions. Make sure there are enough nodes and resources in your cluster to schedule this command pod.\n\n> [!NOTE]\n> The output for `command invoke` is limited to 512kB in size.",
        "vector": [
            -0.012864679098129272,
            -0.03675263375043869,
            0.00032110634492710233,
            -0.006448050960898399,
            -0.0006689224392175674,
        ],
    }

    result = search_client.upload_documents(documents=document)

    print("Upload of new document succeeded: {}".format(result[0].succeeded))

In [139]:
upload_sample_document()

Upload of new document succeeded: True


In [89]:
from azure.search.documents import SearchIndexingBufferedSender

# Upload some documents to the index
def upload_documents_batch(directory_path):
    with open(directory_path, 'r') as file:  
        documents = json.load(file)  
    
    # Use SearchIndexingBufferedSender to upload the documents in batches optimized for indexing  
    with SearchIndexingBufferedSender(  
        endpoint=azure_search_service_endpoint,  
        index_name=index_name,  
        credential=credential,  
    ) as batch_client:  
        # Add upload actions for all documents  
        batch_client.upload_documents(documents=documents)  
    print(f"Uploaded {len(documents)} documents in total")  

In [90]:
upload_documents_batch('./data/chunks/')

PermissionError: [Errno 13] Permission denied: './data/chunks/'

In [91]:
def upload_documents():
    result = search_client.upload_documents(documents=chunks)

    for doc in result:
        print("Upload of new document succeeded: {}".format(doc.succeeded))

In [92]:
upload_documents()

Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document succeeded: True
Upload of new document su

In [46]:
def simple_analyze_text():
    # [START simple_analyze_text]
    from azure.core.credentials import AzureKeyCredential
    from azure.search.documents.indexes import SearchIndexClient
    from azure.search.documents.indexes.models import AnalyzeTextOptions

    search_index_client = SearchIndexClient(search_service_endpoint, credential)

    analyze_request = AnalyzeTextOptions(text="One's <two/>", analyzer_name="standard.lucene")

    result = search_index_client.analyze_text(index_name, analyze_request)
    print(result.as_dict())

In [104]:
simple_analyze_text()

{'tokens': [{'token': "one's", 'start_offset': 0, 'end_offset': 5, 'position': 0}, {'token': 'two', 'start_offset': 7, 'end_offset': 10, 'position': 1}]}


## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [93]:
from azure.search.documents.models import VectorizedQuery

# Pure Vector Search
query = "tools for software development"  

embedding = get_embeddings_from_azure_text_embedding(query)
# embedding = client.embeddings.create(input=query, model=embedding_model_name, dimensions=1536).data[0].embedding
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["page_title", "page_date", "chunk_title", "chunk_content"],
)  
  
for result in results:
    print(f"Page Date: {result['page_date']}")  
    print(f"Page Title: {result['page_title']}")  
    print(f"Chunk Title: {result['chunk_title']}")  
    print(f"Chunk Content: {result['chunk_content']}")
    print(f"Score: {result['@search.score']}")  


[-0.04127116873860359, -0.017787490040063858, -0.025318343192338943, 0.006818829104304314, 0.03216774761676788, 0.03672819584608078, 0.008522444404661655, 0.04410179331898689, -0.033618006855249405, -0.01176804956048727, 0.01421426609158516, -0.03367042541503906, 0.008352082222700119, -0.004241564776748419, 0.013786178082227707, 0.05664739012718201, -0.018433989956974983, -0.023728301748633385, -0.03159113973379135, -0.041760414838790894, -0.005434095393866301, -0.022767288610339165, -0.03180081769824028, -0.01611008495092392, -0.0071639204397797585, -0.0020126684103161097, 0.030001100152730942, 0.001161952968686819, -0.027519937604665756, 0.009898440912365913, 0.002256198087707162, -0.014196793548762798, -0.005569511093199253, -0.005071531515568495, 0.02729278802871704, -0.012038880959153175, -0.00960140023380518, 0.0036780615337193012, -0.0010134326294064522, -0.01668669283390045, -0.007675004657357931, -0.030577708035707474, -0.03414219617843628, 0.0033853892236948013, -0.0027519937